In [79]:
# This is the test script for the final submission 
# No external library other than numpy is allowed 

# sys is only used to point to the correct folder
import sys
sys.path.insert(0, 'project1/scripts')

# import all the functions from the helper module 
from proj1_helpers import *
from implementations import *
# numpy has been included in the proj1_helpers module
# Use np directly 

In [67]:
# sub_sample: take only first 50 data points 
yb, input_data, ids = load_csv_data("test.csv", sub_sample=False)


In [68]:
# yb: class label (s=1, b=-1)
print("The dimension of yb is", yb.shape)
print(yb)
# Features 
print("The dimension of input data is", input_data.shape)
print(input_data)
# ids: event ids  
print("The dimension of ids is", ids.shape)
print(ids)

The dimension of yb is (568238,)
[1. 1. 1. ... 1. 1. 1.]
The dimension of input data is (568238, 30)
[[-999.      79.589   23.916 ... -999.    -999.       0.   ]
 [ 106.398   67.49    87.949 ... -999.    -999.      47.575]
 [ 117.794   56.226   96.358 ... -999.    -999.       0.   ]
 ...
 [ 108.497    9.837   65.149 ... -999.    -999.       0.   ]
 [  96.711   20.006   66.942 ... -999.    -999.      30.863]
 [  92.373   80.109   77.619 ... -999.    -999.       0.   ]]
The dimension of ids is (568238,)
[350000 350001 350002 ... 918235 918236 918237]


In [74]:
w = np.load('weights.npy')

In [75]:
features = [1, 5, 8, 10, 11, 12, 13, 17, 22, 25]

In [81]:
tx = build_poly(input_data[:,features], 1)

y_pred = predict_labels(w, tx)

In [82]:
create_csv_submission(ids, y_pred, "prediction.csv")